# Параллельное программирование

## 1. О параллелизме

**Параллельные вычисления** — способ решать задачи с помощью нескольких вычислительных модулей. Распараллеливать можно независимые части в задаче или её решении.

Выделяют два вида параллелизма.
- *Параллелизм задач* — когда есть независимы подзадачи. Разные инструкции выполняются на одних и тех же данных.
- *Параллелизм данных*, когда есть независимые участки данных. Одинаковые инструкции выполняются на разных кусочках данных.

#### Имеет ли смысл распараллеливать задачу? Закон Амдала (англ. Amdahl’s Law)
Закон Амдала показывает во сколько раз меньше времени потребуется параллельной программе для решения задачи по сравнению с последовательной. 
$$
\text{Speed-up}=\frac{1}{S+P/n}
$$

### Модели параллельного программирования


Модель программирования — теоретическая абстракция.

#### Модель с общей памятью (англ. Shared Memory Model)

Различают два вида:
- С потоками (англ. threads). OpenMP, Posix Threads.
- Без потоков.

#### Модель с распределенной памятью (англ. Distributed Memory Model)

Модель передачи сообщений (англ. Message Passing Model MPI).

- Асинхронная передача сообщений. Actor model
- Синхронная передача сообщений. Communicating sequential processes

### Способы синхронизации

- Семафоры
- Мьютекс (англ. mutex = mutual exclusion) aka Lock.
- Критические секции

### Типичные проблемы

## 2. GIL (Global Interpreter Lock)

GIL — глобальный lock на все ресурсы. Это особенность реализации CPython.

## 3. Threads

## 4. Processes

## 5. MPI

## Бонус: аппаратный взгляд на паралелизм

### Классификация Флина

[Классификация Флина](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)

![Flynn's Taxonomy](img/parallel-architectures.png)